In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp
!pip install streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
spark = sparknlp.start()

In [ ]:
### Select the model and re-run all the cells below ####
MODEL_NAME='norne_6B_100'
model_dict={'norne_840B_300': 'glove_840B_300',
            'norne_6B_100' : 'glove_100d',
            'norne_6B_300': 'glove_6B_300'
}

In [ ]:
## Generating Example Files ##

text_list = ["""William Henry Gates III (født 28. oktober 1955) er en amerikansk forretningsmagnat, programvareutvikler, investor og filantrop. Han er mest kjent som medgründer av Microsoft Corporation. I løpet av sin karriere hos Microsoft hadde Gates stillingene som styreleder, administrerende direktør (CEO), president og sjef programvarearkitekt, samtidig som han var den største individuelle aksjonæren fram til mai 2014. Han er en av de mest kjente gründere og pionerene i mikrodatarevolusjon på 1970- og 1980-tallet. Han er født og oppvokst i Seattle, Washington, og grunnla Microsoft sammen med barndomsvennen Paul Allen i 1975, i Albuquerque, New Mexico; det fortsatte å bli verdens største programvare for datamaskinprogramvare. Gates ledet selskapet som styreleder og administrerende direktør til han gikk av som konsernsjef i januar 2000, men han forble styreleder og ble sjef for programvarearkitekt. I løpet av slutten av 1990-tallet hadde Gates blitt kritisert for sin forretningstaktikk, som har blitt ansett som konkurransedyktig. Denne uttalelsen er opprettholdt av en rekke dommer. I juni 2006 kunngjorde Gates at han skulle gå over til en deltidsrolle hos Microsoft og på heltid ved Bill & Melinda Gates Foundation, den private veldedige stiftelsen som han og kona, Melinda Gates, opprettet i 2000. [ 9] Han overførte gradvis arbeidsoppgavene sine til Ray Ozzie og Craig Mundie. Han trakk seg som styreleder for Microsoft i februar 2014 og tiltrådte et nytt verv som teknologirådgiver for å støtte den nyutnevnte administrerende direktøren Satya Nadella.""",
 """Mona Lisa er et oljemaleri fra 1500-tallet skapt av Leonardo. Det holdes på Louvre i Paris.""", 
]


In [ ]:
!pip install googletrans


In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

lang = 'xx'
if model_dict[MODEL_NAME] == 'glove_100d':
  lang='en'
embeddings = WordEmbeddingsModel.pretrained(model_dict[MODEL_NAME], lang=lang).\
                    setInputCols(["document", 'token']).\
                    setOutputCol("embeddings")

public_ner = NerDLModel.pretrained(MODEL_NAME, 'no') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["document", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[ documentAssembler, 
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
norne_6B_100 download started this may take some time.
Approximate size to download 14.5 MB
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-------------------------------+---------+
|chunk                          |ner_label|
+-------------------------------+---------+
|William Henry Gates III        |PER      |
|Microsoft Corporation          |ORG      |
|Microsoft                      |ORG      |
|Gates                          |PER      |
|Seattle                        |GPE_LOC  |
|Washington                     |GPE_LOC  |
|Microsoft                      |ORG      |
|Paul Allen                     |PER      |
|Albuquerque                    |GPE_LOC  |
|New Mexico                     |GPE_LOC  |
|Gates                          |PER      |
|Gates                          |PER      |
|Gates                          |PER      |
|Microsoft                      |ORG      |
|Bill & Melinda Gates Foundation|ORG      |
|Melinda Gates                  |PER      |
|Ray Ozzie                      |PER      |
|Craig Mundie                   |PER      |
|Han                            |PER      |
|Microsoft                      